# Create Person Table

In [ ]:
CREATE TABLE dbo.people
  (
     person_id         INT IDENTITY(1, 1) PRIMARY KEY,
     person_firstname  NVARCHAR(50) NULL,
     person_lastname   NVARCHAR(50) NULL,
     person_email      NVARCHAR(255) NULL,
     person_address    NVARCHAR(100) NULL,
     person_city       NVARCHAR(50) NULL,
     person_province   NVARCHAR(2) NULL,
     person_postalcode NVARCHAR(7) NULL
  );

# Create Stored Procedure
This stored procedure will receive input from an Azure Function and INSERT a new row into the [dbo].[people] table.

In [ ]:
CREATE PROCEDURE [dbo].[AddNewPerson]
(
    @FirstName  nvarchar(50)    = NULL,
    @LastName   nvarchar(50)    = NULL,
    @Email      nvarchar(255)   = NULL,
    @Address    nvarchar(100)   = NULL,
    @City       nvarchar(50)    = NULL,
    @Province   nvarchar(2)     = NULL,
    @PostalCode nvarchar(7)     = NULL
    )
    AS
        BEGIN
        BEGIN TRY
            BEGIN TRANSACTION INSERT_NEW_PERSON
                /*  
                 Try to insert the new record
                 into the [dbo].[people] table
                 */
                INSERT INTO [dbo].[people]
                (
                    person_firstname,
                    person_lastname,
                    person_email,
                    person_address,
                    person_city,
                    person_province,
                    person_postalcode
                    )
                    VALUES
                    (
                    @FirstName,
                    @LastName,
                    @Email,
                    @Address,
                    @City,
                    @Province,
                    @PostalCode
                    );
                COMMIT TRANSACTION INSERT_NEW_PERSON
        END TRY
    BEGIN CATCH
        IF (XACT_STATE()) = -1
        BEGIN
            /*
                If the trasaction fails to commit, 
                Roll it back and report the error.
            */
            ROLLBACK TRANSACTION INSERT_NEW_PERSON;
        END;
        SELECT  
            ERROR_NUMBER() AS ErrorNumber  
            ,ERROR_SEVERITY() AS ErrorSeverity  
            ,ERROR_STATE() AS ErrorState  
            ,ERROR_PROCEDURE() AS ErrorProcedure  
            ,ERROR_LINE() AS ErrorLine  
            ,ERROR_MESSAGE() AS ErrorMessage;  
    END CATCH
END;
GO

# Security
Add the Azure AD Security Group as a User

Create the database Role with EXECUTE permission on the [dbo].[AddNewPerson] Stored Procedure

Add the Azure AD Security Group as a Member of the Rol

In [1]:
-- Grant access to the database to the Azure AD Security Group
CREATE USER [PeopleDB_ManagedIdentities] FROM EXTERNAL PROVIDER;
GO

-- Add the role for the Managed Identities User Group
CREATE ROLE [PersonCreator_ManagedIdentities];
GO

-- Allow the role to EXECUTE the [dbo].[AddNewPerson] stored procedure
GRANT EXECUTE ON OBJECT::[dbo].[AddNewPerson] TO [PersonCreator_ManagedIdentities];
GO

-- Add the Managed Identities Security Group as Member of the Role
ALTER ROLE [PersonCreator_ManagedIdentities] ADD MEMBER [PeopleDB_ManagedIdentities];
GO


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.646